In [40]:
%matplotlib widget

%load_ext cython

In [2]:
import numpy as np
import pandas as pd
from scipy.signal import butter, sosfiltfilt
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

import inertial_sensor_routines as isr
import signal_features as sf
# import PfyMU

In [63]:
%%cython
# cython: infer_types = True
# cython: boundscheck = False
# cython: wraparound = False
from numpy import fft, sum as nsum, less_equal, zeros, conjugate, argmax, real, array
cimport cython
from libc.math cimport log, pow, exp, floor, ceil
from signal_features._extensions.common cimport mean_1d


cdef double gmean(const double[:] x):
    cdef Py_ssize_t n = x.size, i
    cdef double logsum = 0.0, prod = 1.0
    cdef double large = 1.e64, small=1.e-64

    for i in range(n):
        prod *= x[i]
        if (prod > large) or (prod < small):
            logsum += log(prod)
            prod = 1.

    return exp((logsum + log(prod)) / n)


cpdef linspace(double start, double stop, int N):
    cdef double[:] arr = zeros(N)
    cdef double step = (stop - start) / N
    cdef Py_ssize_t i

    for i in range(N):
        arr[i] = i * step + start

    return arr


def frequency_features_2d(const double[:, :] x, double fs, double low_cut, double hi_cut):
    # local variables
    cdef Py_ssize_t N=x.shape[0], P=x.shape[1], j, k
    cdef double invlog2 = 1 / log(2.)
    cdef double mean
    # return variables
    max_freq = zeros(P)
    max_freq_val = zeros(P)
    dom_freq_ratio = zeros(P)
    spectral_flatness = zeros(P)
    spectral_entropy = zeros(P)

    cdef double[:] maxf = max_freq, maxfv = max_freq_val
    cdef double[:] domf_ratio = dom_freq_ratio
    cdef double[:] spec_flat = spectral_flatness, spec_ent = spectral_entropy

    # function
    cdef int nfft = 2 ** (<int>(log(N) * invlog2))
    cdef double[:] freq = linspace(0.0, 0.5 * fs, nfft)
    print(array(freq))

    cdef int ihcut = <int>(floor(hi_cut / (fs / 2) * (nfft - 1)) + 1)  # high cutoff
    cdef int ilcut = <int>(ceil(low_cut / (fs / 2) * (nfft - 1)))  # low cutoff
    if ihcut > nfft:
        ihcut = <int>(nfft)


    cdef complex[:, :] sp_hat = fft.fft(x, 2 * nfft, axis=0)
    cdef double[:, :] sp_norm = real(sp_hat[ilcut:ihcut, :]
                                     * conjugate(sp_hat[ilcut:ihcut, :]))
    sp_norm = sp_norm / nsum(sp_norm, axis=0, keepdims=True) + 1e-10

    imax = argmax(sp_norm, axis=0)
    print(imax)
    for k in range(P):
        maxf[k] = freq[imax[k] + ilcut]
        maxfv[k] = sp_norm[imax[k] + ilcut, k]

        # spectral flatness
        mean = 0.
        mean_1d(sp_norm[:, k], &mean)
        spec_flat[k] = 10. * log(gmean(sp_norm[:, k]) / mean) / log(10.)

    for j in range(ihcut - ilcut):
        for k in range(P):
            # dominant frequency ratio
            if ((maxf[k] - 0.5) < freq[j] < (maxf[k] + 0.5)):
                domf_ratio[k] += sp_norm[j, k]
            # spectral entropy estimate
            logps = log(sp_norm[j, k]) * invlog2
            spec_ent[k] -= logps * sp_norm[j, k]
    for k in range(P):
        spec_ent[k] /= log(ihcut - ilcut) * invlog2

    return max_freq, max_freq_val, dom_freq_ratio, spectral_flatness, spectral_entropy

In [64]:
domfreq, dfv, *_ = frequency_features_2d(y.reshape((-1, 1)), 2000/10, 0.0, 60.0)
print(domfreq, dfv)

[0.00000000e+00 9.76562500e-02 1.95312500e-01 ... 9.97070312e+01
 9.98046875e+01 9.99023438e+01]
[10]
[0.9765625] [0.79181985]


In [68]:
nfft = 2 ** int((np.log10(x.size) / np.log10(2)))
print(nfft, np.fft.fftfreq(nfft * 2, d=10/2000)[:12])

1024 [0.         0.09765625 0.1953125  0.29296875 0.390625   0.48828125
 0.5859375  0.68359375 0.78125    0.87890625 0.9765625  1.07421875]


In [17]:
x = np.linspace(0, 10, 2000)

y = np.sin(2 * np.pi * x) + 0.2 * np.sin(15 * np.pi * x) + 0.07 * np.sin(100 * np.pi * x)

In [70]:
sp_hat = np.abs(np.fft.fft(y, n=2048))
frq = np.fft.fftfreq(2048, d=10/2000)

"""
cdef complex[:, :] sp_hat = fft.fft(x, 2 * nfft, axis=0)
    cdef double[:, :] sp_norm = real(sp_hat[ilcut:ihcut, :]
                                     * conjugate(sp_hat[ilcut:ihcut, :]))
    sp_norm = sp_norm / nsum(sp_norm, axis=0, keepdims=True) + 1e-10
"""
sp_norm = np.real(sp_hat * np.conjugate(sp_hat))
sp_norm = sp_norm / (np.sum(sp_norm, axis=0, keepdims=True) + 1e-10)

In [71]:
f, (ax, ax1) = plt.subplots(2, figsize=(10, 5))
ax.plot(x, y)
ax1.plot(frq[:int(frq.size//2)], sp_hat[:int(frq.size//2)])
axx1 = ax1.twinx()
axx1.plot(frq[:int(frq.size//2)], sp_norm[:int(frq.size//2)], '--', color='C1')

f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
domfreq, dfv, *_ = sf.dominant_frequency(y, 10/2000, low_cutoff=0.0, high_cutoff=60.0)
print(domfreq, dfv)

[2.44140625e-05] [0.79181636]
